In [ ]:
!pip install openai gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.7 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
import traceback
from contextlib import redirect_stdout
import io

from google.colab import userdata


# ✅ Initialize OpenAI Client
client = OpenAI(api_key=userdata.get('OPENAI'))

def generate_code(query, error_feedback=None):
    """Generate or refine code using LLM"""
    if error_feedback:
        base_prompt = (
            "You are a senior Python engineer.\n\n"
            f"Task:\n{query}\n\n"
            f"Previous error:\n{error_feedback}\n"
            "Fix the code accordingly.\n"
            "Write clean Python code only, inside triple backticks.\n"
            "Ensure you add a sample test case at the end."
        )
    else:
        base_prompt = (
            "You are a senior Python engineer.\n\n"
            f"Task:\n{query}\n\n"
            "Write clean Python code only, inside triple backticks.\n"
            "Ensure you add a sample test case at the end."
        )

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": base_prompt}],
        temperature=0.1
    )
    content = response.choices[0].message.content
    # Extract code block safely
    if "```" in content:
        code = content.split("```")[1].strip()
        if code.startswith("python"):
            code = code[len("python"):].strip()
    else:
        code = content
    return code

def safe_execute(code):
    """Safely execute code and return output or error"""
    f = io.StringIO()
    try:
        with redirect_stdout(f):
            exec_globals = {}
            exec(code, exec_globals)
        return True, f.getvalue()
    except Exception:
        return False, traceback.format_exc()
    finally:
        f.close()

def save_code_to_file(code, filename="generated_code.py"):
    """Save the successful code to a file"""
    with open(filename, "w") as file:
        file.write(code)
    print(f"\n💾 Code saved to `{filename}`")

def auto_chain_agent(query, max_retries=3):
    """Main AutoChain agent logic with auto-save"""
    code = generate_code(query)

    for attempt in range(1, max_retries + 1):
        print(f"\n🚀 Attempt {attempt}")
        print("Generated Code:\n")
        print(f"```python\n{code}\n```")

        success, result = safe_execute(code)
        print("\nExecution Result:\n")
        print(result)

        if success:
            print(f"\n✅ Success in attempt {attempt}")
            save_code_to_file(code)
            break
        else:
            print(f"\n🔄 Error occurred, refining code...")
            code = generate_code(query, error_feedback=result)
    else:
        print("\n❌ Failed after retries. Please review the last code and error.")

# ---------------- Example Usage ----------------
query = "Write a function to calculate the factorial of a number and test it with 5"
auto_chain_agent(query)



🚀 Attempt 1
Generated Code:

```python
def factorial(n):
    """Calculate the factorial of a non-negative integer n."""
    if n < 0:
        raise ValueError("Factorial is not defined for negative numbers.")
    if n == 0 or n == 1:
        return 1
    result = 1
    for i in range(2, n + 1):
        result *= i
    return result

# Sample test case
if __name__ == "__main__":
    test_number = 5
    print(f"The factorial of {test_number} is {factorial(test_number)}")
```

Execution Result:



✅ Success in attempt 1

💾 Code saved to `generated_code.py`


In [ ]:
from openai import OpenAI
import traceback
from contextlib import redirect_stdout
import io
import re

from google.colab import userdata

# ✅ Initialize OpenAI Client
client = OpenAI(api_key=userdata.get('OPENAI'))

def generate_code(query, error_feedback=None):
    """Generate or refine code using LLM"""
    if error_feedback:
        base_prompt = (
            "You are a senior Python engineer.\n\n"
            f"Task:\n{query}\n\n"
            f"Previous error:\n{error_feedback}\n"
            "Fix the code accordingly.\n"
            "Ensure to include proper assert statements in the test case to check correctness.\n"
            "Write clean Python code only, inside triple backticks.\n"
        )
    else:
        base_prompt = (
            "You are a senior Python engineer.\n\n"
            f"Task:\n{query}\n\n"
            "Ensure to include proper assert statements in the test case to check correctness.\n"
            "Write clean Python code only, inside triple backticks.\n"
        )

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": base_prompt}],
        temperature=0.1
    )
    content = response.choices[0].message.content
    # Extract code block safely
    if "```" in content:
        code = content.split("```")[1].strip()
        if code.startswith("python"):
            code = code[len("python"):].strip()
    else:
        code = content
    return code

def safe_execute(code):
    """Safely execute code and count test cases"""
    f = io.StringIO()
    passed, failed = 0, 0
    try:
        with redirect_stdout(f):
            exec_globals = {}
            exec(code, exec_globals)
    except AssertionError as ae:
        return False, f"❌ AssertionError: {ae}", passed, failed + 1
    except Exception as e:
        return False, f"❌ Exception: {traceback.format_exc()}", passed, failed + 1
    else:
        # Attempt to count assert statements as naive test count
        test_count = len(re.findall(r"assert ", code))
        passed = test_count
        return True, f.getvalue() + f"\n✅ {test_count} Test case(s) passed.", passed, failed
    finally:
        f.close()

def save_code_to_file(code, filename="generated_code.py"):
    """Save the successful code to a file"""
    with open(filename, "w") as file:
        file.write(code)
    print(f"\n💾 Code saved to `{filename}`")

def auto_chain_agent(query, max_retries=3):
    """Main AutoChain agent logic with test reporting and auto-save"""
    code = generate_code(query)

    for attempt in range(1, max_retries + 1):
        print(f"\n🚀 Attempt {attempt}")
        print("Generated Code:\n")
        print(f"```python\n{code}\n```")

        success, result, passed, failed = safe_execute(code)
        print("\nExecution Result:\n")
        print(result)

        if success:
            print(f"\n✅ Success in attempt {attempt} - Passed: {passed}, Failed: {failed}")
            save_code_to_file(code)
            break
        else:
            print(f"\n🔄 Error occurred, refining code...")
            code = generate_code(query, error_feedback=result)
    else:
        print("\n❌ Failed after retries. Please review the last code and error.")

# ---------------- Example Usage ----------------
query = """
Write a function to calculate the factorial of a number.
Test it with 5 and make sure the result is 120.
Include assert statements in your test case.
"""
auto_chain_agent(query)



🚀 Attempt 1
Generated Code:

```python
def factorial(n):
    """Calculate the factorial of a non-negative integer n."""
    if n < 0:
        raise ValueError("Factorial is not defined for negative numbers.")
    if n == 0 or n == 1:
        return 1
    result = 1
    for i in range(2, n + 1):
        result *= i
    return result

# Test case
def test_factorial():
    result = factorial(5)
    assert result == 120, f"Expected 120, but got {result}"

# Run the test
test_factorial()
```

Execution Result:


✅ 1 Test case(s) passed.

✅ Success in attempt 1 - Passed: 1, Failed: 0

💾 Code saved to `generated_code.py`


In [ ]:
query = """
A constraint satisfaction problem (CSP) is a problem where we need to find an assignment of values to variables that satisfies a set of constraints.
The CSP can be represented as a graph where nodes represent variables and edges represent constraints.

a) Define the CSP problem and explain its application in real-world scenarios.
b) Implement a Python class to represent the CSP graph, including methods to add variables, constraints, and check satisfiability.
c) Implement a constraint propagation algorithm (e.g., arc consistency) to optimize the CSP solving process.
d) Extend your solution to support weighted constraints and explain how to handle conflicts between constraints.
e) Analyze the performance of your implementation using a benchmark dataset and provide recommendations for optimization.

The CSP problem is a fundamental problem in artificial intelligence and has applications in various domains, such as scheduling, planning, and optimization.
The constraint propagation algorithm is a crucial component of CSP solving, as it helps to eliminate inconsistent assignments and reduce the search space.
Weighted constraints introduce an additional layer of complexity and require careful conflict resolution strategies.
"""
auto_chain_agent(query)


🚀 Attempt 1
Generated Code:

```python
from collections import defaultdict, deque
from typing import Any, Dict, List, Tuple, Callable

class CSP:
    def __init__(self):
        self.variables = set()
        self.domains = {}
        self.constraints = defaultdict(list)
        self.weights = {}

    def add_variable(self, variable: str, domain: List[Any]):
        self.variables.add(variable)
        self.domains[variable] = domain

    def add_constraint(self, var1: str, var2: str, constraint: Callable[[Any, Any], bool], weight: float = 1.0):
        self.constraints[var1].append((var2, constraint))
        self.constraints[var2].append((var1, constraint))
        self.weights[(var1, var2)] = weight
        self.weights[(var2, var1)] = weight

    def is_satisfiable(self, assignment: Dict[str, Any]) -> bool:
        for var in assignment:
            for neighbor, constraint in self.constraints[var]:
                if neighbor in assignment and not constraint(assignment[var], assi

In [ ]:
from collections import defaultdict, deque
from typing import Any, Dict, List, Tuple, Callable

class CSP:
    def __init__(self):
        self.variables = set()
        self.domains = {}
        self.constraints = defaultdict(list)
        self.weights = {}

    def add_variable(self, variable: str, domain: List[Any]):
        self.variables.add(variable)
        self.domains[variable] = domain

    def add_constraint(self, var1: str, var2: str, constraint: Callable[[Any, Any], bool], weight: float = 1.0):
        self.constraints[var1].append((var2, constraint))
        self.constraints[var2].append((var1, constraint))
        self.weights[(var1, var2)] = weight
        self.weights[(var2, var1)] = weight

    def is_satisfiable(self, assignment: Dict[str, Any]) -> bool:
        for var in assignment:
            for neighbor, constraint in self.constraints[var]:
                if neighbor in assignment and not constraint(assignment[var], assignment[neighbor]):
                    return False
        return True

    def arc_consistency(self):
        queue = deque((var1, var2) for var1 in self.variables for var2, _ in self.constraints[var1])
        while queue:
            var1, var2 = queue.popleft()
            if self.revise(var1, var2):
                if not self.domains[var1]:
                    return False
                for neighbor, _ in self.constraints[var1]:
                    if neighbor != var2:
                        queue.append((neighbor, var1))
        return True

    def revise(self, var1: str, var2: str) -> bool:
        revised = False
        for x in self.domains[var1][:]:
            if not any(self.constraints[var1][0][1](x, y) for y in self.domains[var2]):
                self.domains[var1].remove(x)
                revised = True
        return revised

    def solve(self) -> Dict[str, Any]:
        if not self.arc_consistency():
            return {}
        return self.backtrack({})

    def backtrack(self, assignment: Dict[str, Any]) -> Dict[str, Any]:
        if len(assignment) == len(self.variables):
            return assignment

        unassigned_vars = [v for v in self.variables if v not in assignment]
        var = min(unassigned_vars, key=lambda v: len(self.domains[v]))

        for value in self.domains[var]:
            local_assignment = assignment.copy()
            local_assignment[var] = value
            if self.is_satisfiable(local_assignment):
                result = self.backtrack(local_assignment)
                if result:
                    return result

        return {}

# Test cases
def test_csp():
    csp = CSP()
    csp.add_variable('A', [1, 2, 3])
    csp.add_variable('B', [1, 2, 3])
    csp.add_variable('C', [1, 2, 3])

    csp.add_constraint('A', 'B', lambda a, b: a != b)
    csp.add_constraint('B', 'C', lambda b, c: b != c)
    csp.add_constraint('A', 'C', lambda a, c: a != c)

    solution = csp.solve()
    assert solution['A'] != solution['B']
    assert solution['B'] != solution['C']
    assert solution['A'] != solution['C']

    print("All test cases passed.")

test_csp()

All test cases passed.


In [ ]:

from openai import OpenAI
import traceback
from contextlib import redirect_stdout
import io
import re

from google.colab import userdata

# ✅ Initialize OpenAI Client
client = OpenAI(api_key=userdata.get('OPENAI'))

# ---------------- AutoChain Agent ----------------
def generate_code(query, error_feedback=None):
    if error_feedback:
        base_prompt = (
            "You are a senior Python engineer.\n\n"
            f"Task:\n{query}\n\n"
            f"Previous error:\n{error_feedback}\n"
            "Fix the code accordingly.\n"
            "Ensure to include assert statements in the test case to verify correctness.\n"
            "Write clean Python code only, inside triple backticks.\n"
        )
    else:
        base_prompt = (
            "You are a senior Python engineer.\n\n"
            f"Task:\n{query}\n\n"
            "Ensure to include assert statements in the test case to verify correctness.\n"
            "Write clean Python code only, inside triple backticks.\n"
        )

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": base_prompt}],
        temperature=0.1
    )
    content = response.choices[0].message.content
    if "```" in content:
        code = content.split("```")[1].strip()
        if code.startswith("python"):
            code = code[len("python"):].strip()
    else:
        code = content
    return code

def safe_execute(code):
    f = io.StringIO()
    passed, failed = 0, 0
    try:
        with redirect_stdout(f):
            exec_globals = {}
            exec(code, exec_globals)
    except AssertionError as ae:
        return False, f"❌ AssertionError: {ae}", passed, failed + 1
    except Exception as e:
        return False, f"❌ Exception: {traceback.format_exc()}", passed, failed + 1
    else:
        test_count = len(re.findall(r"assert ", code))
        passed = test_count
        return True, f.getvalue() + f"\n✅ {test_count} Test case(s) passed.", passed, failed
    finally:
        f.close()

def save_to_file(content, filename):
    with open(filename, "w") as file:
        file.write(content)
    print(f"\n💾 Saved to `{filename}`")

def auto_chain_agent(query, max_retries=3):
    code = generate_code(query)

    for attempt in range(1, max_retries + 1):
        print(f"\n🚀 Attempt {attempt}")
        print("Generated Code:\n")
        print(f"```python\n{code}\n```")

        success, result, passed, failed = safe_execute(code)
        print("\nExecution Result:\n")
        print(result)

        if success:
            print(f"\n✅ Success in attempt {attempt} - Passed: {passed}, Failed: {failed}")
            save_to_file(code, "generated_code.py")
            return code, result  # Return code for chaining
        else:
            print(f"\n🔄 Error occurred, refining code...")
            code = generate_code(query, error_feedback=result)
    else:
        print("\n❌ Failed after retries. Please review the last code and error.")
        save_to_file(code, "generated_code.py")
        return code, result  # Return last attempt anyway

# ---------------- CodeReview Agent ----------------
def code_review_agent(query, code):
    review_prompt = f"""
You are an expert Python code reviewer and educator.

Task Description:
------------------------------
{query}

Generated Code:
------------------------------
```python
{code}
Please carefully review the code and rate it on:

Correctness (0-10)
Completeness (0-10)
Code Quality (0-10)
Robustness (0-10)
Efficiency (0-10)
Provide a markdown table with the metrics and the scores.
Then, give a short but critical justification for each metric.
"""
response = client.chat.completions.create(
model="gpt-4o",
messages=[{"role": "user", "content": review_prompt}],
temperature=0.2
)
review_content = response.choices[0].message.content
print("\n📋 Code Review Report:\n")
print(review_content)
save_to_file(review_content, "review_report.md")
return review_content

---------------- Chained Agent ----------------

def chained_code_agent(query):
# Step 1: Generate and execute
code, exec_result = auto_chain_agent(query)

# Step 2: Review the generated code
review_content = code_review_agent(query, code)

# Combined output
print("\n✅ Chained Agent Completed\n")
return code, exec_result, review_content
---------------- Example Usage ----------------

query = """
Write a function to calculate the factorial of a number.
Test it with 5 and make sure the result is 120.
Include assert statements in your test case.
"""
chained_code_agent(query)

In [ ]:
from openai import OpenAI
import traceback
from contextlib import redirect_stdout
import io
import re

from google.colab import userdata

# ✅ Initialize OpenAI Client
client = OpenAI(api_key=userdata.get('OPENAI'))

# ---------------- AutoChain Agent ----------------
def generate_code(query, error_feedback=None):
    if error_feedback:
        base_prompt = (
            "You are a senior Python engineer.\n\n"
            f"Task:\n{query}\n\n"
            f"Previous error:\n{error_feedback}\n"
            "Fix the code accordingly.\n"
            "Ensure to include assert statements in the test case to verify correctness.\n"
            "Write clean Python code only, inside triple backticks.\n"
        )
    else:
        base_prompt = (
            "You are a senior Python engineer.\n\n"
            f"Task:\n{query}\n\n"
            "Ensure to include assert statements in the test case to verify correctness.\n"
            "Write clean Python code only, inside triple backticks.\n"
        )

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": base_prompt}],
        temperature=0.1
    )
    content = response.choices[0].message.content
    if "```" in content:
        code = content.split("```")[1].strip()
        if code.startswith("python"):
            code = code[len("python"):].strip()
    else:
        code = content
    return code

def safe_execute(code):
    f = io.StringIO()
    passed, failed = 0, 0
    try:
        with redirect_stdout(f):
            exec_globals = {}
            exec(code, exec_globals)
    except AssertionError as ae:
        return False, f"❌ AssertionError: {ae}", passed, failed + 1
    except Exception as e:
        return False, f"❌ Exception: {traceback.format_exc()}", passed, failed + 1
    else:
        test_count = len(re.findall(r"assert ", code))
        passed = test_count
        return True, f.getvalue() + f"\n✅ {test_count} Test case(s) passed.", passed, failed
    finally:
        f.close()

def save_to_file(content, filename):
    with open(filename, "w") as file:
        file.write(content)
    print(f"\n💾 Saved to `{filename}`")

def auto_chain_agent(query, max_retries=3):
    code = generate_code(query)

    for attempt in range(1, max_retries + 1):
        print(f"\n🚀 Attempt {attempt}")
        print("Generated Code:\n")
        print(f"```python\n{code}\n```")

        success, result, passed, failed = safe_execute(code)
        print("\nExecution Result:\n")
        print(result)

        if success:
            print(f"\n✅ Success in attempt {attempt} - Passed: {passed}, Failed: {failed}")
            save_to_file(code, "generated_code.py")
            return code, result  # Return code for chaining
        else:
            print(f"\n🔄 Error occurred, refining code...")
            code = generate_code(query, error_feedback=result)
    else:
        print("\n❌ Failed after retries. Please review the last code and error.")
        save_to_file(code, "generated_code.py")
        return code, result  # Return last attempt anyway

# ---------------- CodeReview Agent ----------------
def code_review_agent(query, code):
    review_prompt = f"""
You are an expert Python code reviewer and educator.

Task Description:
------------------------------
{query}

Generated Code:
------------------------------
```python
{code}
```

Please carefully review the code and rate it on:

Correctness (0-10)
Completeness (0-10)
Code Quality (0-10)
Robustness (0-10)
Efficiency (0-10)

Provide a markdown table with the metrics and the scores.
Then, give a short but critical justification for each metric.
"""
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": review_prompt}],
        temperature=0.2
    )
    review_content = response.choices[0].message.content
    print("\n📋 Code Review Report:\n")
    print(review_content)
    save_to_file(review_content, "review_report.md")
    return review_content

# ---------------- Chained Agent ----------------
def chained_code_agent(query):
    # Step 1: Generate and execute
    code, exec_result = auto_chain_agent(query)

    # Step 2: Review the generated code
    review_content = code_review_agent(query, code)

    # Combined output
    print("\n✅ Chained Agent Completed\n")
    return code, exec_result, review_content

# ---------------- Example Usage ----------------
if __name__ == "__main__":
    query = """
    Write a function to calculate the factorial of a number.
    Test it with 5 and make sure the result is 120.
    Include assert statements in your test case.
    """
    chained_code_agent(query)


🚀 Attempt 1
Generated Code:

```python
def factorial(n):
    """Calculate the factorial of a non-negative integer n."""
    if n < 0:
        raise ValueError("Factorial is not defined for negative numbers.")
    if n == 0 or n == 1:
        return 1
    result = 1
    for i in range(2, n + 1):
        result *= i
    return result

# Test case
def test_factorial():
    result = factorial(5)
    assert result == 120, f"Expected 120, but got {result}"

# Run the test
test_factorial()
```

Execution Result:


✅ 1 Test case(s) passed.

✅ Success in attempt 1 - Passed: 1, Failed: 0

💾 Saved to `generated_code.py`

📋 Code Review Report:

| Metric       | Score |
|--------------|-------|
| Correctness  | 10    |
| Completeness | 10    |
| Code Quality | 9     |
| Robustness   | 9     |
| Efficiency   | 10    |

**Justification:**

- **Correctness (10/10):** The function correctly calculates the factorial of a non-negative integer. It handles edge cases like 0 and 1 correctly, returning 1 as